In [1]:
import pandas as pd
import numpy as np
import sys
import os
import re
import hydrant.ZonalProcessing.ZonalProcessing as zp

In [2]:
# inputs
file_path = '/home/shg096/scratch/test/gistool_outputs/'

## Soil file preparation for soil class

In [3]:
# read the soil file, get it majority and save
file_name_csv = 'soil_classes/domain_stats_usda_mode_soilclass_250m_ll.csv' # GIS produced file to read
modified_file_name_csv = 'soil_classes/modified_domain_stats_usda_mode_soilclass_250m_ll.csv' # modified csv to save
modified_file_name_nc = 'soil_classes/modified_domain_stats_usda_mode_soilclass_250m_ll.nc' # modified nc to save

# read soil file
df = pd.read_csv(file_path+file_name_csv)
# dictionary for the csv file
info = {'ID': 'COMID',
        'name_of_dataset': 'soil_class',
        'explanation_of_dataset': '',
        'prefix': 'frac'}
# Mapping dictionary
mapping = {
    1: 'clay',
    2: 'silty clay',
    3: 'sandy clay',
    4: 'clay loam',
    5: 'silty clay loam',
    6: 'sandy clay loam',
    7: 'loam',
    8: 'silty loam',
    9: 'sandy loam',
    10: 'silt',
    11: 'loamy sand',
    12: 'sand'
}


# call the function
ds_modified, df_modified = zp.Fraction_to_Xarray(df, info, mapping)

# saved the modified
df_modified.to_csv(file_path+modified_file_name_csv)
ds_modified.to_netcdf(file_path+modified_file_name_nc, mode='w')

ds_modified

<xarray.Dataset>
Dimensions:              (COMID: 5, soil_class: 5)
Coordinates:
  * COMID                (COMID) int64 28070338 28070572 ... 28072168 28072213
  * soil_class           (soil_class) int64 2 3 8 11 12
Data variables:
    soil_class_majority  (COMID) int64 2 3 2 2 3
    soil_class_frac      (COMID, soil_class) float64 0.6928 0.2773 ... 0.0 0.0
    soil_class_names     (soil_class) <U10 'silty clay' 'sandy clay' ... 'sand'

# Landcover preparation for the MODIS

In [4]:
# read the soil file, get it majority and save
file_name_csv = 'MODIS/MCD12Q1.006/domain_stats_MCD12Q1.006_2001.csv' # GIS produced file to read
modified_file_name_csv = 'MODIS/MCD12Q1.006/modified_domain_stats_MCD12Q1.006_2001.csv' # modified csv to save
modified_file_name_nc = 'MODIS/MCD12Q1.006/modified_domain_stats_MCD12Q1.006_2001.nc' # modified nc to save



df = pd.read_csv(file_path+file_name_csv)

info = {'ID': 'COMID',
        'name_of_dataset': 'land_cover_NALCMS',
        'explanation_of_dataset': '',
        'prefix': 'frac'}

mapping   =  {1: 'Evergreen Needleleaf Forests',\
              2: 'Evergreen Broadleaf Forests',\
              3: 'Deciduous Needleleaf Forests',\
              4: 'Deciduous Broadleaf Forests',\
              5: 'Mixed Forests',\
              6: 'Closed Shrublands',\
              7: 'Open Shrublands',\
              8: 'Woody Savannas',\
              9: 'Savannas',\
              10: 'Grasslands',\
              11: 'Permanent Wetlands',\
              12: 'Croplands',\
              13: 'Urban and Built-up Lands',\
              14: 'Cropland/Natural Vegetation Mosaics',\
              15: 'Permanent Snow and Ice',\
              16: 'Barren',\
              17: 'Water Bodies'}

# call the function
ds_modified, df_modified = zp.Fraction_to_Xarray(df, info, mapping)

# saved the modified
df_modified.to_csv(file_path+modified_file_name_csv)
ds_modified.to_netcdf(file_path+modified_file_name_nc, mode='w')

ds_modified


<xarray.Dataset>
Dimensions:                     (COMID: 5, land_cover_NALCMS: 7)
Coordinates:
  * COMID                       (COMID) int64 28070338 28070572 ... 28072213
  * land_cover_NALCMS           (land_cover_NALCMS) int64 4 5 7 8 9 10 12
Data variables:
    land_cover_NALCMS_majority  (COMID) int64 12 10 10 10 7
    land_cover_NALCMS_frac      (COMID, land_cover_NALCMS) float64 0.02859 .....
    land_cover_NALCMS_names     (land_cover_NALCMS) <U27 'Deciduous Broadleaf...

# Elevation

In [5]:
# read the soil file, get it majority and save
file_name_csv = 'merit_hydro/domain_stats_elv.csv' # csv to read
file_name_nc = 'merit_hydro/domain_stats_elv.nc' # file to save


df = pd.read_csv(file_path+file_name_csv)

info = {'ID': 'COMID'}

# Mapping dictionary
mapping = {'mean':{'units':'m'},
           'max':{'units':'m'},
           'min':{'units':'m'},
           'median':{'units':'m'}}

ds = zp.Stat_to_Xarray(df, info, mapping)

# save
ds.to_netcdf(file_path+file_name_nc, mode = 'w')

ds

<xarray.Dataset>
Dimensions:  (COMID: 5)
Coordinates:
  * COMID    (COMID) int64 28070338 28070572 28072028 28072168 28072213
Data variables:
    min      (COMID) float64 324.0 916.7 1.026e+03 1.026e+03 916.8
    max      (COMID) float64 1.375e+03 1.293e+03 2.161e+03 2.03e+03 1.859e+03
    mean     (COMID) float64 874.1 1.06e+03 1.599e+03 1.37e+03 1.227e+03
    median   (COMID) float64 916.6 1.048e+03 1.674e+03 1.273e+03 1.177e+03

# 1D combination of MODIS land cover and soil class

In [6]:
# call the function to create the merger of soil and land cover
soil_type_name_csv = 'soil_classes/modified_domain_stats_usda_mode_soilclass_250m_ll.csv'
land_cover_name_csv = 'MODIS/MCD12Q1.006/modified_domain_stats_MCD12Q1.006_2001.csv'

soil_type = pd.read_csv(file_path+soil_type_name_csv)
land_cover = pd.read_csv(file_path+land_cover_name_csv)

result,report, ds =  zp.intersect_df(soil_type, land_cover,
                                     df_mappings={'df1': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'soil_class'}, 
                                                  'df2': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'land_cover_NALCMS'}},
                                     output_dim={'id':'COMID'},
                                     threshold = 0.05,
                                     remove_zero_combinations = True)
# save
os.makedirs(file_path+'comb/') if not os.path.exists(file_path+'comb/') else None
result.to_csv(file_path+'comb/'+'modified_West_Combination_frac.csv')
report.to_csv(file_path+'comb/'+'modified_West_Combination_report.csv')
ds.to_netcdf(file_path+'comb/'+'modified_West_Combination.nc', mode='w')

ds

The indexes of all DataFrames are exactly the same with the same order.
total number of non zero combinations:  6


<xarray.Dataset>
Dimensions:            (comb: 6, COMID: 5)
Coordinates:
  * comb               (comb) object 'comb_0001' 'comb_0002' ... 'comb_0006'
  * id                 (COMID) int64 28070338 28070572 ... 28072168 28072213
Dimensions without coordinates: COMID
Data variables:
    comb_explanation   (comb) object 'soil_class2 land_cover_NALCMS7' ... 'so...
    soil_class         (comb) int64 2 2 2 3 3 3
    land_cover_NALCMS  (comb) int64 7 10 12 7 10 12
    comb_int           (comb) int64 1 2 3 4 5 6
    fraction           (COMID, comb) float64 0.0 0.2574 0.3107 ... 0.1732 0.0
    max_fraction       (COMID) object 'comb_0003' 'comb_0005' ... 'comb_0004'